# Análisis Completo de Regresión Lineal

Este notebook muestra paso a paso cómo realizar un análisis de **Regresión Lineal** empleando un conjunto de datos que contiene:
- **valuationdate**: fecha de valoración (opcional para el modelo)
- **revenues**: ingresos
- **ebit**: utilidad antes de intereses e impuestos
- **depreciation**: depreciación
- **capex**: gastos de capital
- **ncwc**: cambio en capital de trabajo neto
- **bv**: (posiblemente) valor en libros
- **y**: variable objetivo que deseamos predecir (por ejemplo, un valor financiero o de valoración)

Iremos desde la **carga** y **exploración** del dataset hasta la **interpretación** de coeficientes y la **evaluación** de métricas importantes.

## Paso 0: Importar librerías y configuración

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ajustes de estilo
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)

## Paso 1: Carga y Exploración de Datos (EDA)

En este primer bloque, cargaremos nuestro dataset y realizaremos un Análisis Exploratorio básico.

In [ ]:
# Carga del dataset
# Ajusta el nombre 'mi_dataset.csv' según tu archivo real.

df = pd.read_csv("mi_dataset.csv")  # Reemplazar con el archivo correcto
df.head()

In [ ]:
# Forma y tipos de datos
print("Forma del DataFrame:", df.shape)
df.info()

### Estadísticas Descriptivas y Distribuciones

In [ ]:
df.describe()

In [ ]:
# Ejemplo: Histogramas y/o Boxplots de las variables numéricas
variables_numericas = ["revenues", "ebit", "depreciation", "capex", "ncwc", "bv", "y"]

for col in variables_numericas:
    plt.figure()
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribución de {col}")
    plt.show()

# Boxplot general
plt.figure()
sns.boxplot(data=df[variables_numericas])
plt.title("Boxplot de Variables Numéricas")
plt.show()

### Preguntas de Reflexión (Exploración)
1. ¿Qué tipo de problema estás intentando resolver al usar `y` como variable objetivo?
2. ¿Notas valores atípicos (outliers) en alguna de las variables? ¿Cómo podría afectar a la regresión lineal?
3. ¿Crees que la columna `valuationdate` podría aportar información para predecir `y`, o la excluirías del análisis?

_Responde con tu razonamiento propio aquí._

## Paso 2: Preparación y Limpieza de Datos

1. **Manejo de valores faltantes**.
2. **Posible eliminación** o **imputación** de outliers.
3. **Transformaciones** (escalado, log, etc.) si son necesarias.

In [ ]:
# Revisión de valores faltantes
df.isna().sum()

Si encuentras valores faltantes, decide si:
- Rellenarlos (imputar) con media, mediana, o un método más robusto.
- Eliminarlos por completo si son pocos y no afectan significativamente la muestra.

### Opcional: Transformaciones logarítmicas
Si, por ejemplo, `revenues` o `y` tienen una escala muy grande y sospechas que la relación puede ser más lineal en el **espacio logarítmico**, podrías crear columnas como:
```python
df['log_revenues'] = np.log(df['revenues'] + 1)
df['log_y'] = np.log(df['y'] + 1)
```
y usar esas variables en la regresión.

### Pregunta de Reflexión (Limpieza)
¿Por qué es relevante tratar valores faltantes y/o outliers **antes** de entrenar el modelo de regresión lineal?

_Responde con tu razonamiento aquí._

## Paso 3: Análisis de Correlación (Opcional, pero Recomendado para Múltiples Variables)
Si pretendes usar más de una variable independiente (e.g. `revenues`, `ebit`, `depreciation`, etc.), es útil verificar:
- **Relación** de cada variable con `y`.
- **Multicolinealidad** entre variables predictoras.

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Matriz de Correlación")
plt.show()

### Pregunta de Reflexión (Correlación)
Si detectas que dos columnas (por ejemplo, `revenues` y `ebit`) están **muy** correlacionadas, ¿cómo podría afectar esto a la interpretación de tus coeficientes?

_Responde con tu razonamiento._

## Paso 4: Selección de Variables e Implementación del Modelo

1. **Seleccionar** cuáles columnas usarás como variables explicativas (features). Puede que uses todas o hagas una selección.
2. **Asignar** `X` (features) y `y` (variable objetivo).
3. **Dividir** en conjunto de entrenamiento (train) y conjunto de prueba (test).

In [ ]:
# Supongamos que decides usar 'revenues', 'ebit', 'depreciation', 'capex', 'ncwc', 'bv'
# como variables predictoras. Ajusta según tus hallazgos.

features = ["revenues", "ebit", "depreciation", "capex", "ncwc", "bv"]
X = df[features]
y = df["y"]  # Variable objetivo

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Tamaño de entrenamiento:", X_train.shape[0])
print("Tamaño de prueba:", X_test.shape[0])

### Preguntas de Reflexión (Selección de Variables)
1. ¿Por qué es importante separar los datos en train y test?
2. ¿Qué pasaría si entrenas y evalúas en el **mismo** conjunto de datos?

_Responde con tu razonamiento._

## Paso 5: Entrenar el Modelo de Regresión Lineal
Usaremos la clase `LinearRegression` de scikit-learn.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

print("Intercept (beta_0):", model.intercept_)
print("Coeficientes (beta_i):", model.coef_)

### Interpretación de Coeficientes
Si la lista de features es `["revenues", "ebit", "depreciation", "capex", "ncwc", "bv"]` y tu `model.coef_` es algo como `[a, b, c, d, e, f]`, entonces:

\[ y \approx \beta_0 + a\cdot revenues + b\cdot ebit + c\cdot depreciation + d\cdot capex + e\cdot ncwc + f\cdot bv \]

Cada coeficiente indica cómo cambia `y` en promedio **por cada unidad** que se incrementa la variable (manteniendo las demás constantes).

## Paso 6: Evaluación y Validación del Modelo
Usaremos las siguientes métricas:
- **MSE (Mean Squared Error)**
- **MAE (Mean Absolute Error)**
- **R² (Coeficiente de Determinación)**

In [ ]:
# Predicciones
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Métricas en entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Métricas en prueba
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("=== Entrenamiento ===")
print(f"MSE: {mse_train:.2f}")
print(f"MAE: {mae_train:.2f}")
print(f"R²: {r2_train:.2f}")

print("\n=== Prueba ===")
print(f"MSE: {mse_test:.2f}")
print(f"MAE: {mae_test:.2f}")
print(f"R²: {r2_test:.2f}")

### Preguntas de Reflexión (Evaluación)
1. ¿Por qué comparamos las métricas en **Entrenamiento** vs **Prueba**?
2. ¿Qué significa si el error en entrenamiento es **muy** distinto (mucho menor o mucho mayor) que en prueba?
3. ¿Cómo interpretar un **R²** cercano a 1, cercano a 0 o incluso negativo?

_Responde con tu razonamiento._

## Paso 7: Visualización de Resultados
Para un análisis adicional, podemos observar:
1. La **recta/hiperplano** de regresión (fácil en regresión simple, más complejo en múltiple).
2. Los **residuos** (y - y_pred).

### Ejemplo: Gráfico de Residuos vs. Predicciones
Si se trata de una regresión múltiple, un gráfico muy común es `residuos vs. y_pred` para ver si los errores están bien distribuidos.

In [ ]:
residuos_test = y_test - y_test_pred

plt.figure()
sns.scatterplot(x=y_test_pred, y=residuos_test)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Residuos vs. Predicción (Conjunto de Prueba)")
plt.xlabel("Predicción")
plt.ylabel("Residuos")
plt.show()

### Pregunta de Reflexión (Residuos)
Si observas un patrón claro en la gráfica de residuos (por ejemplo, forma de U), ¿qué podría indicar respecto a las **asunciones** de la regresión lineal?

_Responde con tu razonamiento._

## Paso 8: Interpretación y Conclusiones

1. **Revisa** los coeficientes: ¿son consistentes con la lógica de negocio? (p.ej., si `ebit` aumenta, esperas que `y` aumente o disminuya?).
2. **Observa** el nivel de error y R²: ¿el modelo es suficientemente bueno para su uso práctico?
3. **Considera** la posibilidad de:
   - Añadir más variables.
   - Aplicar **regresiones polinómicas** (X², X³) si sospechas no linealidad.
   - Usar **regularización** (Ridge, Lasso) si hay multicolinealidad o se requieren coeficientes más estables.

### Preguntas de Reflexión Final
1. ¿Crees que la Regresión Lineal Simple/Múltiple **captura adecuadamente** las relaciones en tus datos?
2. ¿Qué otras técnicas (ej. árboles de decisión, bosques aleatorios, etc.) considerarías para comparar resultados?
3. ¿El dataset es suficientemente grande y variado para hacer conclusiones robustas?

_Responde con tu razonamiento._

## Resumen
En este Notebook hemos:
1. **Explorado** los datos (EDA) y detectado posibles outliers.
2. **Limpieza** de datos y transformaciones opcionales.
3. **Seleccionado** variables y dividido en train/test.
4. **Entrenado** un modelo de Regresión Lineal.
5. **Evaluado** con MSE, MAE, R² y analizado residuos.
6. **Reflexionado** sobre los coeficientes y conclusiones.

¡Has completado un **análisis de regresión lineal** de forma integral! Para afianzar, puedes:
- Ajustar parámetros (aunque la regresión lineal simple en sklearn no tiene muchos).
- Comparar con otros modelos.
- Realizar validación cruzada en lugar de una simple división train/test.

¡Éxitos en tu proyecto de análisis!